In [56]:
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

from constants import *
from utils import black_video_region, shorten_video
from visualize import overlay_keypoints

if MMPOSE_FOLDER not in sys.path:
  sys.path.insert(0, MMPOSE_FOLDER)

import numba

import hpe


In [57]:
%reload_ext autoreload
%autoreload 2

In [58]:
video_path = f"{INTERIM_FOLDER}extracted/S1/Videos/Directions 1.54138969.mp4"
short_path = f"{INTERIM_FOLDER}short.mp4"
shorten_video(video_path=video_path, output_path=short_path, start=0, end=0.5)


Moviepy - Building video /media/joris/uca-msc-dsai/research_project/data/interim/short.mp4.
Moviepy - Writing video /media/joris/uca-msc-dsai/research_project/data/interim/short.mp4



Moviepy - Done !
Moviepy - video ready /media/joris/uca-msc-dsai/research_project/data/interim/short.mp4


In [59]:
import cv2
import plotly.express as px

black_path = f"{INTERIM_FOLDER}black.mp4"
h_prop = 0.35

cap = cv2.VideoCapture(short_path)

first_round = True
success, frame_bgr = cap.read()
while success:
  if first_round:
    h, w, _ = frame_bgr.shape
    pt1 = (0, 0)
    pt2 = (w, int(h_prop * h))
    
    out = cv2.VideoWriter(black_path, cv2.VideoWriter_fourcc(*"mp4v"), 50, (w, h))
    first_round = False


  cv2.rectangle(
    img=frame_bgr,
    pt1=pt1,
    pt2=pt2,
    color=(0, 0, 0),
    thickness=-1,
  )

  out.write(frame_bgr)
  
  success, frame_bgr = cap.read()

out.release()
cap.release()


In [61]:
video_path = f"{DATA_FOLDER}extracted/S1/Videos/Directions 1.54138969.mp4"
video_path = f"{INTERIM_FOLDER}short.mp4"

hpe.predict(video_path=black_path)


Stage 1: 2D pose detection.
Initializing model...
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Running 2D pose detection inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 1.1 task/s, elapsed: 23s, ETA:     0s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
<class 'mmpose.models.detectors.pose_lifter.PoseLifter'>
Running 2D-to-3D pose lifting inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 6.1 task/s, elapsed: 4s, ETA:     0s


In [28]:
video_path=f"{RAW_FOLDER}s1/S1/Videos/Phoning 1.55011271.mp4",
kp_path=f"{RAW_FOLDER}s1/Poses_D2_Positions_S1/S1/MyPoseFeatures/D2_Positions/Phoning 1.55011271.cdf",
figs = overlay_keypoints(
  video_path=video_path,
  kp_path=kp_path,
  max_frame=1000,
  show_nth_frame=400,
)
for fig in figs:
  fig.show()
